In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [2]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load the Excel file
file_path = '/content/FAQ_list_PAN_TAN_All.xlsx'
df = pd.read_excel(file_path)

# Initialize an empty DataFrame to store the cleaned data
cleaned_data = []

# Iterate over the rows in the DataFrame
for index, row in df.iterrows():
    questions = row['Question'].split('\n')  # Split the questions if there are multiple in one cell
    answer = row['Answer'].strip()  # Get the corresponding answer

    for question in questions:
        question = question.strip()  # Clean up any extra whitespace
        if question:  # Ensure the question is not empty
            # Append the cleaned question and answer to the list
            cleaned_data.append({'Question': question, 'Answer': answer})

# Convert the list to a DataFrame
cleaned_df = pd.DataFrame(cleaned_data)

# Save the cleaned dataset to a new file
cleaned_df.to_csv('./cleaned_dataset.csv', index=False)

# Reformat the dataset into the model's required format
with open('./formatted_output.txt', 'w', encoding='utf-8') as f:
    for index, row in cleaned_df.iterrows():
        formatted_text = f"<s>[INST] {row['Question']} [/INST] {row['Answer']} </s>\n"
        f.write(formatted_text)

In [4]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "distilbert/distilgpt2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = getattr(torch, "float16"), bnb_4bit_quant_type = "nf4"))
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at distilbert/distilgpt2 and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "distilbert/distilgpt2", trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [16]:
# Load the dataset from the formatted text file
dataset = load_dataset('text', data_files='./formatted_output.txt')

training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps = 5000, save_steps=250, logging_steps=100, learning_rate=1e-5)

In [17]:
sft_trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset['train'],
    tokenizer=tokenizer,
    peft_config=LoraConfig(task_type="CAUSAL_LM", r=4, lora_alpha=16, lora_dropout=0.1),
    dataset_text_field="text"
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [18]:
sft_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
100,5.007100
200,4.964700
300,4.936800
400,4.857500
500,4.771200
600,4.736100
700,4.611600
800,4.625700
900,4.609700
1000,4.533800


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=5000, training_loss=4.189716455078125, metrics={'train_runtime': 362.9528, 'train_samples_per_second': 55.104, 'train_steps_per_second': 13.776, 'total_flos': 217498596433920.0, 'train_loss': 4.189716455078125, 'epoch': 17.12})

In [14]:
user_prompt = "What is AO code?"  #r-8
text_generation_pipeline = pipeline(task = "text-generation", model = model, tokenizer = tokenizer, max_length = 300)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] What is AO code? [/INST] You need to provide a name and an address of any authorised department you need to receive your PAN for an Application (Application by Application). https://tin.tin.nsdl.com/PAN-pan-paan.html. However all the applications must be provided by official PAN application. </s> PAN details on www.protean-tinpan.org </s> You may contact the centre for PAN or contact the Centre to obtain your name. </s> To ensure registration of PAN without application, contact the centre or contact the Centre for PAN. </s> In other words, contact the centre for PAN and contact the Centre for PAN as per the definition of PAN, PAN or application or contact the Centre for PAN. You should contact the Centre for PAN. </s> Please visit https://www.proteanpan-interacting.com/services/pan-convertean-statement.html#Forms/panTIN-A-FICA (fee) filed annually for PAN status changes. </s> For PAN documents, contact IT IT Centre https://min.proteam-tin.com/services/panFINITC/panMan.html. F

In [15]:
text_generation_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=250,
    do_sample=False,
)

def chatbot():
    print("Chatbot is running... Type 'exit' to stop.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        else:
            # Format the input as required by the model
            formatted_input = f"<s>[INST] {user_input} [/INST] Chatbot: "
            # Generate the response
            response = text_generation_pipeline(formatted_input)
            # Print the generated response
            print("Chatbot:", response[0]['generated_text'].replace(formatted_input, "").strip())

# Run the chatbot
chatbot()


Chatbot is running... Type 'exit' to stop.
You: Where do I get AO details?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: https://tin.tin.nsdl.com/pan/TIN-downloads.html.html to know the details of the application. </s> PAN- PAN details are not required to be used in PAN applications. </s> https://tin.tin.nsdl.com/pantan/documents/PAN-AO-TIN-verification.html to know the details of PAN applications. </s> PAN details are not required to be used in PAN applications. </s> PAN- PAN applications are not required to be used in PAN applications. </s> PAN applications are not required to be used in PAN applications. </s> </s> </s> </s> PAN applications are not required to be used in PAN applications. </s> PAN applications are not required to be used in PAN applications. </s> PAN applications are not required to be used in PAN applications. </s> </s> The application of PAN applications is not required to be used in PAN applications. </s> PAN applications are not required to be used in PAN applications. </s> </
You: PAN card status?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: https://www.protean-tinpan.com/services/pan/pan_register_online_verification_for_verification_for_pan_verification_for_verification_for_verification_for_verification_verification_for_verification_for_verification_for_verification_verification_for_verification_for_verification_verification_for_verification_for_verification_for_verification_for_verification_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_for_verification_verification_for_verification_for_verification_for_verification_for_verification_for_verification_for_verification_verification_for_verification_for_verification_
You: Please provide a list of gazetted officers?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Chatbot: https://www.protean-tinpan.com/services/pan-de-verification/pan-verification-verification-verification-verification-verification-verification-verification-2.html for details. </s> Protean-FCs> https://www.protean-tinpan.com/tin-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-ververification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-verification-
You: exit
Chatbot: Goodbye!
